
![](https://velog.velcdn.com/images/hyejinbeck/post/d9ababf1-f8b7-4e2d-b27a-4c857e6b6c6f/image.png)


![](https://velog.velcdn.com/images/hyejinbeck/post/fe82d86d-8df5-41f4-a0fa-fd93239a36fd/image.png)

![](https://velog.velcdn.com/images/hyejinbeck/post/cab3e8f3-4ae6-40f7-bdcb-2b35b205537d/image.png)

In [218]:
# 초기환경셋팅 (진행후 주석처리함)
# !pip install -r requirements.txt

In [219]:
# 처음 불러오기
import pandas as pd 
data = pd.read_csv('./Dataset/Dataset_03.csv')
data.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female


In [220]:
# q1문제를 위한 원본 복사 
q1 = data.copy()

# 이마폭,이마높이의 비율에 대한 변수 생성
q1['forehead_ratio'] = q1['forehead_width_cm']/q1['forehead_height_cm']

q1.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender,forehead_ratio
0,1,11.8,6.1,1,0,1,1,Male,1.934426
1,0,14.0,5.4,0,0,1,0,Female,2.592593
2,0,11.8,6.3,1,1,1,1,Male,1.873016
3,0,14.4,6.1,0,1,1,1,Male,2.360656
4,1,13.5,5.9,0,0,0,0,Female,2.288136


In [221]:
# 평균, 표준편차 단위 구하기
mean = q1['forehead_ratio'].mean()
std = q1['forehead_ratio'].std()

# 평균으로부터 3 표준편차(std) 밖 구간 
# -3std <---me|an---> +3std
up3std = mean + (3 * std)  # 상위 
down3std = mean - (3*std)  # 하위 

# 구간 내 (이상치) 갯수 
q1['forehead_ratio']< down3std  
q1['forehead_ratio']> up3std

(q1['forehead_ratio']< down3std ) | (q1['forehead_ratio']> up3std)   # or 
out = (q1['forehead_ratio']< down3std ) | (q1['forehead_ratio']> up3std)
out.sum() 

3

In [222]:
# q2문제를 위한 원본 복사
q2 = q1.copy()

# 성별에 따른 forehead_ratio평균(위에 만든변수)구하기 
q2.gender.unique() 
# array(['Male', 'Female'], dtype=object)

q2[q2.gender=='Male']
q2[q2.gender=='Male']['forehead_ratio'] 
group_m = q2[q2.gender=='Male']['forehead_ratio'] # 남자그룹별 평균 

q2[q2.gender=='Female']
q2[q2.gender=='Female']['forehead_ratio'] 
group_f = q2[q2.gender=='Female']['forehead_ratio'] # 여자그룹별 평균

# 통계검정을 위한 함수 불러오기
from scipy.stats import ttest_ind ,  bartlett
# ttest_ind 이표본T검정 = 독립성 
# ttest_rel 이표본T검정 = 관계성 
# bartlett 등분산 검정용 

# 등분산검정 (등분산이다 vs 등분산이 아니다(이분산이다)) 
bartlett(group_m, group_f)

BartlettResult(statistic=213.42228096492104, pvalue=2.4617792693930456e-48)

In [223]:
# T-Test검정
ttest_ind(group_m, group_f, equal_var=False)
# equal_var = True 등분산 
# equal_var = False 이분산 

q2_out = ttest_ind(group_m, group_f, equal_var=False)
q2_out
print(q2_out)

# pvalue가 유의수준0.05보다 작다면 대립가설(등분산이 아니다 = 이분산이다) 
# pvalue=0.002718670239065719 이므로 이분산이다. 

TtestResult(statistic=2.9994984197511543, pvalue=0.002718670239065719, df=4619.829725747954)


In [224]:
# 검정통계량 구하기 
q2_out.statistic

2.9994984197511543

In [225]:
# q3문제를 위한 원본 복사
q3 = data.copy()

# 데이터셋 분리를 위한 함수 
from sklearn.model_selection import train_test_split

# 데이터셋 분리
train_test_split(q3, test_size=0.3, random_state=123)

train, test = train_test_split(q3, test_size=0.3, random_state=123)

In [245]:
len(q3),len(train),len(test)

(5001, 3500, 1501)

In [229]:
# 7개변수 Feautres 정의 (label인 gender만 제외)
features = train.columns.drop('gender')

# 로지스틱회귀분석 함수 
from sklearn.linear_model import LogisticRegression

# 로지스틱회귀분석 모델 
# model = LogisticRegression()

# 조건추가 
# 조건1. penalty = 'L2'
# 조건2. Inverse of Regularization = 10000
# 조건3. solver = newton-cg
# 조건4. seed(random_state) = 123 

# 로지스틱 회귀분석 모델 with 위의 조건 
model = LogisticRegression(
    C = 100000, 
    solver='newton-cg', 
    random_state=123
)

# 로지스틱 회귀분석 학습 
model.fit(train[features], train.gender)

# 로지스틱 회귀분석 예측값 
pred = model.predict(test[features])
pred

array(['Male', 'Male', 'Male', ..., 'Female', 'Male', 'Male'],
      dtype=object)

In [230]:
# (이건 그냥 추가가정) 참고로 만약 확률로 예측한다면? 
pred_pr = model.predict_proba(test[features])
print(pred_pr) # 각각 확률이 구해짐 = 알파벳순 = 여기에서는 array[Female, Male] 순 

[[1.14575198e-02 9.88542480e-01]
 [2.34265850e-05 9.99976573e-01]
 [2.26112367e-01 7.73887633e-01]
 ...
 [9.77223652e-01 2.27763484e-02]
 [2.22174919e-05 9.99977783e-01]
 [3.81061141e-05 9.99961894e-01]]


In [231]:
# 로지스틱 회귀분석 결과 = 성능평가 모듈 함수
from sklearn.metrics import classification_report, precision_score

# 성능평가 (실제값, 예측값)
classification_report(test.gender, pred)

'              precision    recall  f1-score   support\n\n      Female       0.97      0.96      0.96       743\n        Male       0.96      0.97      0.97       758\n\n    accuracy                           0.97      1501\n   macro avg       0.97      0.97      0.97      1501\nweighted avg       0.97      0.97      0.97      1501\n'

In [232]:
# print() 로 출력해야 깔끔하게 보여집니다. 
print(classification_report(test.gender, pred))

              precision    recall  f1-score   support

      Female       0.97      0.96      0.96       743
        Male       0.96      0.97      0.97       758

    accuracy                           0.97      1501
   macro avg       0.97      0.97      0.97      1501
weighted avg       0.97      0.97      0.97      1501



In [233]:
# 초기환경셋팅 (진행후 주석처리함)
# !pip install -r requirements.txt

# 처음 불러오기
import pandas as pd 
data = pd.read_csv('./Dataset/Dataset_03.csv')

# q1문제를 위한 원본 복사 
q1 = data.copy()

# 이마폭,이마높이의 비율에 대한 변수 생성
q1['forehead_ratio'] = q1['forehead_width_cm']/q1['forehead_height_cm']

# 평균, 표준편차 단위 구하기
mean = q1['forehead_ratio'].mean()
std = q1['forehead_ratio'].std()

# 평균으로부터 3 표준편차(std) 밖 구간 
# -3std <---me|an---> +3std
up3std = mean + (3 * std)  # 상위 
down3std = mean - (3*std)  # 하위 

# 구간 내 (이상치) 갯수 
q1['forehead_ratio']< down3std  
q1['forehead_ratio']> up3std

(q1['forehead_ratio']< down3std ) | (q1['forehead_ratio']> up3std)   # or 
out = (q1['forehead_ratio']< down3std ) | (q1['forehead_ratio']> up3std)
out.sum() 
print(f'outlier의 갯수는 {out.sum()}')
# 3 

# q2문제를 위한 원본 복사
q2 = q1.copy()

# 성별에 따른 forehead_ratio평균(위에 만든변수)구하기 
q2.gender.unique() 
# array(['Male', 'Female'], dtype=object)

q2[q2.gender=='Male']
q2[q2.gender=='Male']['forehead_ratio'] 
group_m = q2[q2.gender=='Male']['forehead_ratio'] # 남자그룹별 평균 

q2[q2.gender=='Female']
q2[q2.gender=='Female']['forehead_ratio'] 
group_f = q2[q2.gender=='Female']['forehead_ratio'] # 여자그룹별 평균

# 통계검정을 위한 함수 불러오기
from scipy.stats import ttest_ind ,  bartlett
# ttest_ind 이표본T검정 = 독립성 
# ttest_rel 이표본T검정 = 관계성 
# bartlett 등분산 검정용 

# 등분산검정 (등분산이다 vs 등분산이 아니다(이분산이다)) 
bartlett(group_m, group_f)

# T-Test검정
ttest_ind(group_m, group_f, equal_var=False)
# equal_var = True 등분산 
# equal_var = False 이분산 
q2_out = ttest_ind(group_m, group_f, equal_var=False)
q2_out
print(q2_out)

# pvalue가 유의수준0.05보다 작다면 대립가설(등분산이 아니다 = 이분산이다) 
# pvalue=0.002718670239065719 이므로 이분산이다. 

# 검정통계량 구하기 
q2_out.statistic
print(f'검정통계량은 {q2_out.statistic:.3f}')

# q3문제를 위한 원본 복사
q3 = data.copy()

# 데이터셋 분리를 위한 함수 
from sklearn.model_selection import train_test_split

# 데이터셋 분리
train_test_split(data, test_size=0.3, random_state=123)

train, test = train_test_split(data, test_size=0.3, random_state=123)

# 데이터셋 분리후 확인

train # 3500 
test # 1501 
q3.info() # 5001 
# len(data), len(train), len(test)

# 7개변수 Feautres 정의 (label인 gender만 제외)
features = train.columns.drop('gender')

# 로지스틱회귀분석 함수 
from sklearn.linear_model import LogisticRegression

# 로지스틱회귀분석 모델 
# model = LogisticRegression()

# 조건추가 
# 조건1. penalty = 'L2'
# 조건2. Inverse of Regularization = 10000
# 조건3. solver = newton-cg
# 조건4. seed(random_state) = 123 

# 로지스틱 회귀분석 모델 with 위의 조건 
model = LogisticRegression(
    C = 100000, 
    solver='newton-cg', 
    random_state=123
)

# 로지스틱 회귀분석 학습 
model.fit(train[features], train.gender)

# 로지스틱 회귀분석 예측값 
pred = model.predict(test[features])

# (이건 그냥 추가가정) 참고로 만약 확률로 예측한다면? 
pred_pr = model.predict_proba(test[features])

# pred 와 pred_pr 
print(pred) # 각각 레이블로 예측 
print(pred_pr) # 각각 확률이 구해짐 = 알파벳순 = 여기에서는 array[Female, Male] 순 

# 로지스틱 회귀분석 결과 = 성능평가 모듈 함수
from sklearn.metrics import classification_report, precision_score

# 성능평가 (실제값, 예측값)
classification_report(test.gender, pred)
print(classification_report(test.gender, pred))

# Male의 Precision 값은 ? 
print(f'Male의 Precision 값은 0.96입니다.')

outlier의 갯수는 3
TtestResult(statistic=2.9994984197511543, pvalue=0.002718670239065719, df=4619.829725747954)
검정통계량은 2.999
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   long_hair                  5001 non-null   int64  
 1   forehead_width_cm          5001 non-null   float64
 2   forehead_height_cm         5001 non-null   float64
 3   nose_wide                  5001 non-null   int64  
 4   nose_long                  5001 non-null   int64  
 5   lips_thin                  5001 non-null   int64  
 6   distance_nose_to_lip_long  5001 non-null   int64  
 7   gender                     5001 non-null   object 
dtypes: float64(2), int64(5), object(1)
memory usage: 312.7+ KB
['Male' 'Male' 'Male' ... 'Female' 'Male' 'Male']
[[1.14575198e-02 9.88542480e-01]
 [2.34265850e-05 9.99976573e-01]
 [2.26112367e-01 7.73887633e-01]


In [246]:
# 초기설치 
# !pip install -r requirements.txt

# 불러오기 
import pandas as pd 
data = pd.read_csv('./Dataset/Dataset_03.csv')
data.info()

############################ 1번문제를 위한 원본 복사
q1 = data.copy()

# 이마의 폭과 높이 사이의 비율 = 변수생성 
q1['forehead_ratio'] = q1['forehead_width_cm'] / q1['forehead_height_cm']

# 이마의 폭과 높이 사이의 비율을 기준으로, 평균값과 표준편차 구하기 
q1_mean = q1['forehead_ratio'].mean()
q1_std = q1['forehead_ratio'].std()

# 평균으로부터 3 표준편차 밖의 경우 구하기 
up = q1_mean + (3 * q1_std)
down = q1_mean - (3 * q1_std)

# 3 표준편차 기준으로 이상치 
q1['forehead_ratio'] < down 
q1['forehead_ratio'] > up 
(q1['forehead_ratio'] < down ) | (q1['forehead_ratio'] > up )    # or = 하나라도 적용되면 
(q1['forehead_ratio'] < down ) | (q1['forehead_ratio'] > up ).sum()
((q1['forehead_ratio'] < down ) | (q1['forehead_ratio'] > up )).sum()

############################ 2번문제를 위한 원본 복사
q2 = q1.copy()

# 성별의 그룹내 갯수 
q2.gender.unique()        # array(['Male', 'Female'], dtype=object) --> 2개

# 성별에 따른 forehead_ratio평균 각각 그룹 구하기 
q2[q2.gender=='Male']
q2[q2.gender=='Male']['forehead_ratio'] 
group_m = q2[q2.gender=='Male']['forehead_ratio'] # 남자그룹별 평균 

q2[q2.gender=='Female']
q2[q2.gender=='Female']['forehead_ratio'] 
group_f = q2[q2.gender=='Female']['forehead_ratio'] # 여자그룹별 평균

# 그룹내 갯수가 2개면       t-test 검정 
# 그룹내 갯수가 3개이상이면   ANOVA  검정  (2개일때 사용하기도함)

# 통계검정을 위한 함수 불러오기
from scipy.stats import ttest_ind ,  bartlett
# ttest_ind 이표본T검정 = 독립성 
# ttest_rel 이표본T검정 = 관계성 
# bartlett 등분산 검정용 

# 여기에서는 이분산을 가정하고 있으나, 해당 조건이 주어지지 않을 때 등분산 검정부터 먼저 해야함. 
# 검정을 위한 함수 불러오기
from scipy.stats import ttest_ind ,  bartlett
# ttest_ind 이표본T검정 = 독립성 
# ttest_rel 이표본T검정 = 관계성 
# bartlett 등분산 검정용 

# 등분산검정 (등분산이다 vs 등분산이 아니다(이분산이다)) 
bartlett(group_m, group_f)

# T-Test검정
ttest_ind(group_m, group_f, equal_var=False)
# equal_var = True 등분산 
# equal_var = False 이분산 
q2_out = ttest_ind(group_m, group_f, equal_var=False)
q2_out
print(q2_out)

# pvalue가 유의수준0.05보다 작다면 대립가설(등분산이 아니다 = 이분산이다) 
# pvalue=0.002718670239065719 이므로 이분산이다. 

# 검정통계량 구하기 
q2_out.statistic
print(f'검정통계량은 {q2_out.statistic:.3f}')

############################ 3번문제를 위한 원본 복사
q3 = data.copy()

# 데이터셋 분리를 위한 함수 
from sklearn.model_selection import train_test_split

# 데이터셋 분리 조건 
train_test_split(q3, test_size=0.3, random_state=123)

# 데이터셋 분리 진행 
train, test = train_test_split(q3, test_size=0.3, random_state=123)

# 데이터셋 분리 후 확인 
print(len(train),len(test),len(q3))

# 7개변수 Feautres 정의 (label인 gender만 제외)
features = train.columns.drop('gender')

# 로지스틱회귀분석 함수 
from sklearn.linear_model import LogisticRegression

# 조건추가 
# 조건1. penalty = 'L2'
# 조건2. Inverse of Regularization = 10000
# 조건3. solver = newton-cg
# 조건4. seed(random_state) = 123 

# 위의 조건으로 로지스틱회귀분석 모델링 
model = LogisticRegression(
    C = 100000, 
    solver='newton-cg', 
    random_state=123
)

# 로지스틱 회귀분석 학습 
model.fit(train[features], train.gender)

# 로지스틱 회귀분석 예측값 
pred = model.predict(test[features])

# (이건 그냥 추가가정) 참고로 만약 확률로 예측한다면? 
pred_pr = model.predict_proba(test[features])

# pred 와 pred_pr 
print(pred) # 각각 레이블로 예측 
print(pred_pr) # 각각 확률이 구해짐 = 알파벳순 = 여기에서는 array[Female, Male] 순 

# 로지스틱 회귀분석 결과 = 성능평가 모듈 함수
from sklearn.metrics import classification_report, precision_score

# 성능평가 (실제값, 예측값)
classification_report(test.gender, pred)
print(classification_report(test.gender, pred))

# Male의 Precision 값은 ? 
print(f'Male의 Precision 값은 0.96입니다.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   long_hair                  5001 non-null   int64  
 1   forehead_width_cm          5001 non-null   float64
 2   forehead_height_cm         5001 non-null   float64
 3   nose_wide                  5001 non-null   int64  
 4   nose_long                  5001 non-null   int64  
 5   lips_thin                  5001 non-null   int64  
 6   distance_nose_to_lip_long  5001 non-null   int64  
 7   gender                     5001 non-null   object 
dtypes: float64(2), int64(5), object(1)
memory usage: 312.7+ KB
TtestResult(statistic=2.9994984197511543, pvalue=0.002718670239065719, df=4619.829725747954)
검정통계량은 2.999
3500 1501 5001
['Male' 'Male' 'Male' ... 'Female' 'Male' 'Male']
[[1.14575198e-02 9.88542480e-01]
 [2.34265850e-05 9.99976573e-01]
 [2.26112367e-01 7.73887633e-01]


In [ ]:
# 초기설치 및 불러오기 


############################ 1번문제를 위한 원본 복사

# 이마의 폭과 높이 사이의 비율 = 변수생성 


# 이마의 폭과 높이 사이의 비율을 기준으로, 평균값과 표준편차 구하기 


# 평균으로부터 3 표준편차 밖의 경우 구하기 


# 3 표준편차 기준으로 이상치 

############################ 2번문제를 위한 원본 복사

# 성별의 그룹내 갯수 


# 성별에 따른 forehead_ratio평균 각각 그룹 구하기 


# 통계검정을 위한 함수 불러오기


# 여기에서는 이분산을 가정하고 있으나, 해당 조건이 주어지지 않을 때 등분산 검정부터 먼저 해야함. 
# 검정을 위한 함수 불러오기


# 등분산검정 (등분산이다 vs 등분산이 아니다(이분산이다)) 


# T-Test검정


# T-Test검정결과 
# 유의수준0.05보다 pvalue작다면 대립가설(등분산이 아니다 = 이분산이다) 


# 검정통계량 구하기 


############################ 3번문제를 위한 원본 복사


# 데이터셋 분리를 위한 함수 


# 데이터셋 분리 조건 


# 데이터셋 분리 진행 


# 데이터셋 분리 후 확인 


# 7개변수 Feautres 정의 (label인 gender만 제외)


# 로지스틱회귀분석 함수 


# 조건추가 
# 조건1. penalty = 'L2'
# 조건2. Inverse of Regularization = 10000
# 조건3. solver = newton-cg
# 조건4. seed(random_state) = 123 

# 위의 조건으로 로지스틱회귀분석 모델링 


# 로지스틱 회귀분석 학습 


# 로지스틱 회귀분석 예측값 


# (이건 그냥 추가가정) 참고로 만약 확률로 예측한다면? 


# pred 와 pred_pr 의 차이? 
# 각각 레이블로 예측 하는 것은 ? 
# 각각 확률이 구해짐 = 알파벳순 = 여기에서는 array[Female, Male] 순으로 진행되는 것은? 

# 로지스틱 회귀분석 결과 = 성능평가 모듈 함수


# 성능평가 (실제값, 예측값 확인)


# Male의 Precision 값은 ? 
